Transformação e Limpeza de dados:
* Corrigir o schema das nossas colunas 
* Remover os dados faltantes

Nesta aula continuaremos a trabalhar com o arquivo csv da aaula de Exploração de dados:

In [54]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')

df.head()

,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,768805383,0,45,M,3,ensino medio,casado,$60K - $80K,blue,39,5,3,1,"12.691,51","1.144,90",42
1,818770008,0,49,F,5,mestrado,solteiro,menos que $40K,blue,44,6,2,1,"8.256,96","1.291,45",33
2,713982108,0,51,M,3,mestrado,casado,$80K - $120K,blue,36,4,0,1,"3.418,56","1.887,72",20
3,769911858,0,40,F,4,ensino medio,NaN,menos que $40K,blue,34,3,1,4,"3.313,03","1.171,56",20
4,709106358,0,40,M,3,sem educacao formal,casado,$60K - $80K,blue,21,5,0,1,"4.716,22","816,08",28


In [55]:
df.shape

(10127, 16)

Correção de schema: Na etapa de exploração notamos que as colunas limite_credito e valor_transacoes_12m estavam constando como colunas categóricas (dtype = object).

In [56]:
df[['limite_credito','valor_transacoes_12m']].dtypes

limite_credito          object
valor_transacoes_12m    object
dtype: object

In [57]:
df[['limite_credito','valor_transacoes_12m']].head()

,limite_credito,valor_transacoes_12m
0,"12.691,51","1.144,90"
1,"8.256,96","1.291,45"
2,"3.418,56","1.887,72"
3,"3.313,03","1.171,56"
4,"4.716,22","816,08"


Para resolver isso vamos criar umaa função lambda para limpar estes dados.

In [58]:
fn = lambda valor: float(valor.replace(".","").replace(",","."))

vamos testar essa função:

In [59]:
valores_originais = ('12.691,51','8.256,96','3.418,56')
valores_limpos = list(map(fn, valores_originais))
print(valores_originais)
print(valores_limpos)

('12.691,51', '8.256,96', '3.418,56')
[12691.51, 8256.96, 3418.56]


Aplicando a função nas colunas de interesse:

In [60]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)

descrevendo novamente o schema:

In [61]:
df.dtypes

id                           int64
default                      int64
idade                        int64
sexo                        object
dependentes                  int64
escolaridade                object
estado_civil                object
salario_anual               object
tipo_cartao                 object
meses_de_relacionamento      int64
qtd_produtos                 int64
iteracoes_12m                int64
meses_inativo_12m            int64
limite_credito             float64
valor_transacoes_12m       float64
qtd_transacoes_12m           int64
dtype: object

Atributos categóricos

In [62]:
df.select_dtypes('object').describe().transpose()

,count,unique,top,freq
sexo,10127,2,F,5358
escolaridade,8608,5,mestrado,3128
estado_civil,9378,3,casado,4687
salario_anual,9015,5,menos que $40K,3561
tipo_cartao,10127,4,blue,9436


Atributos numéricos:

In [63]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

,count,mean,std,min,25%,50%,75%,max
default,10127.0,0.160660,0.367235,0.00,0.000,0.00,0.000,1.00
idade,10127.0,46.325960,8.016814,26.00,41.000,46.00,52.000,73.00
dependentes,10127.0,2.346203,1.298908,0.00,1.000,2.00,3.000,5.00
meses_de_relacionamento,10127.0,35.928409,7.986416,13.00,31.000,36.00,40.000,56.00
qtd_produtos,10127.0,3.812580,1.554408,1.00,3.000,4.00,5.000,6.00
iteracoes_12m,10127.0,2.455317,1.106225,0.00,2.000,2.00,3.000,6.00
meses_inativo_12m,10127.0,2.341167,1.010622,0.00,2.000,2.00,3.000,6.00
limite_credito,10127.0,8632.440165,9088.788041,1438.00,2555.355,4549.42,11068.175,34516.99
valor_transacoes_12m,10127.0,4404.583047,3397.128078,510.16,2155.825,3899.59,4741.310,18484.93
qtd_transacoes_12m,10127.0,64.858695,23.472570,10.00,45.000,67.00,81.000,139.00


Agora vamos para a remoção de dados faltantes:

In [64]:
df.dropna(inplace=True) 
#utilizando o metodo dropna, estamos excluindo as linhas que não possuem dados 

Vamos analisar

In [65]:
df.shape

(7081, 16)

repare que antes da exclusão das linhas com dados faltantes, este era nosso resultado: (10127, 16)

In [66]:
df[df['default'] == 0].shape

(5968, 16)

In [67]:
df[df['default'] == 1].shape

(1113, 16)

In [69]:
qtd_total,_ = df.shape
qtd_adimplente,_ = df[df['default'] == 0].shape
qtd_inadimplente,_ = df[df['default'] == 1].shape

In [71]:
print(f'A nova proporção de clientes adimplentes ativos é de {round(100 * qtd_adimplente / qtd_total, 2)}%')
print(f'A nova proporção de clientes inadimplentes é de {round(100 * qtd_inadimplente / qtd_total, 2)}%')

A nova proporção de clientes adimplentes ativos é de 84.28%
A nova proporção de clientes inadimplentes é de 15.72%
